In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, MaxPooling2D, Conv2D, Activation
from tensorflow.keras.models import Model
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential

In [2]:
# 加载数据集
DataSet = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = DataSet.load_data()

# 数据预处理
train_images, test_images = train_images / 255.0, test_images / 255.0

In [3]:
class SparseConv2D(layers.Layer):
    def __init__(self, filters, kernel_size, p, **kwargs):
        super(SparseConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.p = tf.Variable(float(p), trainable=False)  # 将 p 转换为浮点数类型
        self.counter = tf.Variable(0, trainable=False, dtype=tf.int32)  # 初始化计数器

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                      initializer='glorot_uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.filters,),
                                    initializer='zeros',
                                    trainable=True)

    @tf.function
    def call(self, inputs, training=None):
        if training:
            mask = tf.random.uniform(shape=(self.filters,), minval=0, maxval=1)
            mask = tf.cast(mask < self.p, dtype=tf.float32)
            mask = tf.reshape(mask, [1, 1, 1, self.filters])
            #self.counter.assign_add(1)  # 更新计数器
            #tf.print("\nP is", self.p)  # 使用 tf.print
        else:
            mask = tf.ones([1, 1, 1, self.filters], dtype=tf.float32) * self.p
    
        sparse_kernel = self.kernel * mask
        conv = tf.nn.conv2d(inputs, sparse_kernel, strides=[1, 1, 1, 1], padding='SAME')
        return tf.nn.bias_add(conv, self.bias)

    @tf.function
    def update_p(self, new_p):
        self.p.assign(float(new_p))  # 使用 assign 更新 tf.Variable 的值，并转换为浮点数
        #tf.print("\nEpoch counter is", self.counter)  # 使用 tf.print
        #tf.print("\nP is", self.p)  # 使用 tf.print

In [4]:
def VGG16(nb_class,input_shape):
    input_ten = Input(shape=input_shape)
    #1
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same')(input_ten)
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    #2
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    #3
    x = tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    #4
    x = tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    #5
    x = tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same')(x)
    x = SparseConv2D(filters=512, kernel_size=3, p=0.9, name='sparse_conv2d_3')(x)
    x = layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    #FC
    x = tf.keras.layers.Flatten()(x)
    x = Dense(4096,activation='relu')(x)
    x = Dense(4096,activation='relu')(x)
    output_ten = Dense(nb_class,activation='softmax')(x)
    model = Model(input_ten,output_ten)
    return model

In [5]:
# 设置参数
nb_class = 100
img_height, img_width = 32, 32
input_shape = (img_height, img_width, 3)

# 创建模型
model_VGG16 = VGG16(nb_class, input_shape)

# 使用较小的学习率和 Adam 优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# 编译模型
model_VGG16.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',  # 如果标签是整数，则使用 sparse_categorical_crossentropy
                    metrics=['accuracy'])

model_VGG16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0     

In [6]:
class UpdatePSparsity(tf.keras.callbacks.Callback):
    def __init__(self, model, sparsity_schedule):
        super(UpdatePSparsity, self).__init__()
        self.model = model
        self.sparsity_schedule = sparsity_schedule

    def on_epoch_end(self, epoch, logs=None):
        for layer_name, new_p in self.sparsity_schedule.items():
            layer = self.model.get_layer(name=layer_name)
            if epoch < len(new_p):
                p_value = new_p[epoch]
            else:
                p_value = new_p[-1]  # Use the last value for epochs beyond the predefined ones
            layer.update_p(p_value)
            #print(f"\nEpoch {epoch + 1}: Updated layer {layer_name} sparsity p to {p_value}")

sparsity_schedule = {
    'sparse_conv2d_3': [0.9]
}

In [ ]:
model_VGG16.fit(train_images, train_labels, 
          epochs=40, 
          batch_size=128, 
          validation_data=(test_images, test_labels),  # 确保validation_data是(x_val, y_val)的元组
          callbacks=[UpdatePSparsity(model_VGG16, sparsity_schedule)])

Epoch 1/40
391/391 [==============================] - 12s 22ms/step - loss: 2.2405 - accuracy: 0.1417 - val_loss: 1.8887 - val_accuracy: 0.2373
Epoch 2/40
391/391 [==============================] - 7s 19ms/step - loss: 1.6566 - accuracy: 0.3484 - val_loss: 1.4855 - val_accuracy: 0.4367
Epoch 3/40
391/391 [==============================] - 7s 18ms/step - loss: 1.3317 - accuracy: 0.5028 - val_loss: 1.2330 - val_accuracy: 0.5496
Epoch 4/40
391/391 [==============================] - 7s 18ms/step - loss: 1.0989 - accuracy: 0.6001 - val_loss: 1.0209 - val_accuracy: 0.6343
Epoch 5/40
391/391 [==============================] - 7s 18ms/step - loss: 0.9452 - accuracy: 0.6610 - val_loss: 0.9805 - val_accuracy: 0.6515
Epoch 6/40
391/391 [==============================] - 7s 18ms/step - loss: 0.8071 - accuracy: 0.7142 - val_loss: 0.8905 - val_accuracy: 0.6885
Epoch 7/40
391/391 [==============================] - 7s 18ms/step - loss: 0.6925 - accuracy: 0.7540 - val_loss: 0.8916 - val_accuracy: 0.691